In [48]:
import requests
from bs4 import BeautifulSoup
import time
import pyautogui
from openpyxl import Workbook, load_workbook  # 수정된 부분
import os

# 사용자 입력
#keyword = pyautogui.prompt("검색어를 입력하세요")
lastpage = int(pyautogui.prompt("몇 페이지까지 크롤링 할까요?"))
file_name = f"04_일반뉴스_암_기사_20250326~20250425.xlsx"

keyword = '크롤링'


# 엑셀 파일 열기 또는 생성
if os.path.exists(file_name):
    wb = load_workbook(file_name)
    ws = wb.active
else:
    wb = Workbook()
    ws = wb.active
    ws.title = '일반뉴스기사'
    # 헤더 추가 (파일이 새로 만들어질 때만 추가)
    ws.append(['제목', '날짜', 'URL', '내용'])

# 열 너비 조절
ws.column_dimensions['A'].width = 60
ws.column_dimensions['B'].width = 30
ws.column_dimensions['C'].width = 60
ws.column_dimensions['D'].width = 150

# 페이지 번호
pageNum = 1
for i in range(1, lastpage * 10, 10):
    print(f"{pageNum}페이지 크롤링중입니다 =================")

    response = requests.get(f"https://search.naver.com/search.naver?where=news&query={keyword}&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds=2025.04.14&de=2025.04.20&news&query={keyword}&start={i}")
    response = requests.get(f"https://search.naver.com/search.naver?where=news&query=%ED%81%AC%EB%A1%A4%EB%A7%81&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds=2025.04.14&de=2025.04.20&news&query=%ED%81%AC%EB%A1%A4%EB%A7%81&start=1")
    html = response.text
    #print(html)
    soup = BeautifulSoup(html, 'html.parser')
    #articles = soup.select("div.info_group")
    articles = soup.select("div.group_news")

    print('article >>> ', articles)

    for article in articles:
        #links = article.select("a.info")
        links = article.select("a.bynlPWBHumGsbotLYK9A.jT1DuARpwIlNAFMacxlu")
        
        print(links)
        #tag = soup.find('a', attrs={'nocr': '1'})

        #print("tag >>> ",tag['href'])

        print("len >>> ", len(links))
        time.sleep(0.7)
        if len(links) >= 2:
            url = links[1].attrs['href']
            print("url >>> ", url)

            # 연예 뉴스는 넘어가고 일반 뉴스만 처리
            if "entertain" not in url and "sports" not in url:  # 연예 및 스포츠 뉴스 제외
            #if True:
                #print(1)
                response = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
                #print(2)
                html = response.text
                #print(html)
                soup_sub = BeautifulSoup(html, 'html.parser')

                title_tag, title, date, content = None, None, None, None

                try:
                    # 일반 뉴스의 태그에 맞춰 데이터 추출
                    # 제목: h2 태그의 ID가 article_title_h2
                    title_tag = soup_sub.find('h2', id='article_title_h2')
                    title = title_tag.text.strip() if title_tag else '제목 없음'
                    print("title >>>", title)
                    #date = soup_sub.select_one("span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME")  # 기사 날짜
                    # 날짜: <time> 태그 중 "발행일"이 포함된 첫 번째 태그
                    time_tags = soup_sub.find_all('time')
                    date = '날짜 없음'
                    for tag in time_tags:
                        if '발행일' in tag.text:
                            date = tag.text.replace('발행일 :', '').strip()
                            break
                    print("date >>>", date)
                    #content = soup_sub.select_one("#newsct_article")  # 기사 내용
                    # 본문: articleBody 안의 <p> 태그들 텍스트를 모두 이어붙이기
                    article_body = soup_sub.find('div', id='articleBody')
                    if article_body:
                        paragraphs = article_body.find_all('p')
                        content = '\n\n'.join(p.get_text(separator=' ', strip=True) for p in paragraphs)
                    else:
                        content = '내용 없음'
                    print("content >>> ", content)

                    if not title or not date or not content:
                        raise Exception("Missing element")

                    print(11)    
                    #content_text = content.get_text(separator="\n") if content else "내용을 찾을 수 없습니다."
                    print(22)
                    #print("=======제목======= \n", title.text.strip() if title else "제목을 찾을 수 없습니다.")
                    print("=======제목======= \n", title if title else "제목을 찾을 수 없습니다.")
                    #print("=======날짜======= \n", date.text if date else "날짜를 찾을 수 없습니다.")
                    print("=======날짜======= \n", date if date else "날짜를 찾을 수 없습니다.")
                    print("=======URL======= \n", url)
                    #print("=======내용======= \n", content_text.strip() if content else "내용을 찾을 수 없습니다")
                    print("=======내용======= \n", content if content else "내용을 찾을 수 없습니다")

                    # 엑셀에 저장
                    ws.append([
                        #title.text.strip() if title else "제목을 찾을 수 없습니다.",
                        title if title else "제목을 찾을 수 없습니다.",
                        #date.text.strip() if date else "날짜를 찾을 수 없습니다.",
                        date if date else "날짜를 찾을 수 없습니다.",
                        url,
                        #content_text.strip() if content else "내용을 찾을 수 없습니다."
                        content if content else "내용을 찾을 수 없습니다."
                    ])
                
                except Exception as e:
                    print(f"Error processing article: {e}")
                    continue

    # 마지막 페이지 여부 확인
    next_button = soup.select_one("a.btn_next")
    if next_button:
        isLastPage = next_button.attrs.get('aria-disabled', None)
        if isLastPage == 'true':
            print("마지막 페이지 입니다.")
            break

    pageNum += 1

# 엑셀 파일 저장
wb.save(file_name)

1페이지 크롤링중입니다 =================
article >>>  [<div class="group_news"><ul class="list_news _infinite_list" data-img-mutation=""><div class="_slog_visible" data-block-id="news/prs_template_v2_news_tab_desk.ts" data-fender-root="true" data-meta-area="nws" data-meta-ssc="tab.news.all" data-slog-container="nws" id="fdr-8ccb5a6966e64c22acb373aed0cc935c"><div class="f3a1bNS1jEYwPx4x9AZd desktop_mode api_subject_bx"><div class="sds-comps-vertical-layout sds-comps-full-layout" data-template-id="layout" data-template-type="vertical"><div class="sds-comps-vertical-layout sds-comps-full-layout fender-news-item-list-tab" data-template-id="layout" data-template-type="vertical"><div class="sds-comps-vertical-layout sds-comps-full-layout Ermefm6A3ilpd9Zvt0OZ"><div class="sds-comps-horizontal-layout sds-comps-full-layout sds-comps-profile type-basic size-lg title-color-g10 AFIg_epz83EqgqI2NN7t" data-sds-comp="Profile"><div class="sds-comps-horizontal-layout sds-comps-inline-layout sds-comps-profile-sou